<a href="https://colab.research.google.com/github/deepampatel/randomnlp/blob/main/IndianLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Oct  3 06:56:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

--2020-10-03 06:56:08--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  82.8MB/s    in 21s     

2020-10-03 06:56:30 (90.6 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [3]:
!tar -xvf dakshina_dataset_v1.0.tar

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.nonblock.sections.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.omit_pages.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/na

In [4]:
!ls

dakshina_dataset_v1.0  dakshina_dataset_v1.0.tar  sample_data


In [5]:
import os
import gc
import glob
import torch
import pickle
import joblib
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [6]:
!pip install tokenizers
!git clone https://github.com/huggingface/transformers
!pip install transformers/.


     |████████████████████████████████| 3.0MB 7.8MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 44854 (delta 0), reused 12 (delta 0), pack-reused 44842
Receiving objects: 100% (44854/44854), 32.18 MiB | 3.57 MiB/s, done.
Resolving deltas: 100% (31107/31107), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 8.6MB/s 
     |████████████████████████████████| 3.0MB 31.4MB/s 
     |████████████████████████████████| 1.1MB 49.3MB/s 
  Created wheel for transformers: filename=transformers-3.3.1-cp36-none-any.whl size=1082350 sha256=e47202854357f34fcb94e1dace393bc0637ad682d6ffc1f8bb6672407353df21
  Stored in directory: /tmp/pip-ephem-wheel-cache-a7_5k5s9/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c6

In [7]:
!gzip -d dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.train.text.sorted.tsv.gz
!gzip -d dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.valid.text.sorted.tsv.gz
!ls
#df= pd.read_csv("mr.wiki-filt.train.text.sorted.tsv",sep="\t",error_bad_lines=False)

def save_txt(input_path, output_path):
  df= pd.read_csv(input_path,sep="\t",error_bad_lines=False)
  df.columns = [x for x in range(len(df.columns))]
  with open(output_path,'w') as file:
    for line in df[6].values:
      file.write(line+'\n')

dakshina_dataset_v1.0  dakshina_dataset_v1.0.tar  sample_data  transformers


In [8]:
save_txt("dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.train.text.sorted.tsv","marathi_train.txt")
save_txt("dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.valid.text.sorted.tsv","marathi_val.txt")

b'Skipping line 18714: expected 7 fields, saw 8\nSkipping line 36613: expected 7 fields, saw 8\nSkipping line 36614: expected 7 fields, saw 8\nSkipping line 36616: expected 7 fields, saw 8\nSkipping line 36618: expected 7 fields, saw 8\nSkipping line 36621: expected 7 fields, saw 8\nSkipping line 36623: expected 7 fields, saw 8\nSkipping line 36626: expected 7 fields, saw 8\nSkipping line 36628: expected 7 fields, saw 8\nSkipping line 36629: expected 7 fields, saw 8\nSkipping line 36631: expected 7 fields, saw 8\nSkipping line 36634: expected 7 fields, saw 8\nSkipping line 36637: expected 7 fields, saw 8\nSkipping line 37022: expected 7 fields, saw 8\nSkipping line 45406: expected 7 fields, saw 8\nSkipping line 47957: expected 7 fields, saw 8\nSkipping line 47961: expected 7 fields, saw 8\nSkipping line 47963: expected 7 fields, saw 8\nSkipping line 47972: expected 7 fields, saw 8\nSkipping line 50169: expected 7 fields, saw 9\nSkipping line 50170: expected 7 fields, saw 9\nSkipping li

In [9]:
!head -3 marathi_train.txt

एखाद्या कुशल चित्रकाराने कुंचल्याच्या अवघ्या चार-सहा फटकाऱ्यांसरशी एखादे सुरेख चित्र निर्माण करावे तद्वत अवघ्या आठ ओळीत बालकवींनी एक सुंदर निसर्गचित्र शब्दांच्या कुंचल्याने या कवितेत रेखाटलेले आहे.  
हे चित्र रंगविताना कवीने विविध रंग वापरलेले आढळतील. 
निळासावळा झरा, शेतमळ्यांची हिरवी गरदी, पांढरी पायवाट व काळा डोह-मोजक्या रंगांनी शब्दांच्या चौकटीत बसविलेले हे एक साधे व जिवंत चित्र आहे.  


In [10]:
!head -3 marathi_val.txt

'म्हणी' म्हणजे लोकांनी वारंवार उच्चारलेले वाक्य.
म्हणीमध्ये जीवनातील विशिष्ट अनुभव, माहिती, सत्य व उपदेश साठवलेला असतो. 
जगातील सर्व प्रमुख भाषांमध्ये म्हणी भरपूर प्रमाणात आढळतात.


In [11]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ["marathi_train.txt"]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model(".", "esperberto")

['./esperberto-vocab.json', './esperberto-merges.txt']

In [12]:
!mkdir martbertochar
tokenizer.save_model("martbertochar")

['martbertochar/vocab.json', 'martbertochar/merges.txt']

In [13]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./martbertochar/vocab.json",
    "./martbertochar/merges.txt",
)

In [14]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [15]:
op = tokenizer.encode("एखाद्या कुशल चित्रकाराने कुंचल्याच्या अवघ्या चार-सहा फटकाऱ्यांसरशी एखादे सुरेख चित्र निर्माण करावे तद्वत अवघ्या आठ ओळीत बालकवींनी एक सुंदर निसर्गचित्र शब्दांच्या कुंचल्याने या कवितेत रेखाटलेले आहे.  ")

In [16]:
from tokenizers.decoders import ByteLevel
decoder = ByteLevel()

In [17]:
tokenizer.decode(op)

TypeError: ignored

In [18]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [19]:
from transformers import RobertaTokenizerFast


tokenizer = RobertaTokenizerFast.from_pretrained("./martbertochar", max_len=512)

In [20]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [21]:
model.num_parameters()
# => 84 million parameters

83504416

In [22]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="marathi_train.txt",
    block_size=128,
)

CPU times: user 1min 2s, sys: 2.56 s, total: 1min 5s
Wall time: 38.8 s


In [23]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [24]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./martbertochar",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)


/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:267: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [25]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{'loss': 4.9067041015625, 'learning_rate': 4.923780487804878e-05, 'epoch': 0.06097560975609756, 'total_flos': 2020331226046464, 'step': 500}
{'loss': 4.063455078125, 'learning_rate': 4.847560975609756e-05, 'epoch': 0.12195121951219512, 'total_flos': 4034762364076032, 'step': 1000}
{'loss': 3.6450576171875, 'learning_rate': 4.771341463414634e-05, 'epoch': 0.18292682926829268, 'total_flos': 6053265845465088, 'step': 1500}
{'loss': 3.335876953125, 'learning_rate': 4.695121951219512e-05, 'epoch': 0.24390243902439024, 'total_flos': 8070903553069056, 'step': 2000}
{'loss': 3.0396875, 'learning_rate': 4.618902439024391e-05, 'epoch': 0.3048780487804878, 'total_flos': 10092549472641024, 'step': 2500}
{'loss': 2.863130859375, 'learning_rate': 4.542682926829269e-05, 'epoch': 0.36585365853658536, 'total_flos': 12105216997404672, 'step': 3000}
{'loss': 2.711384765625, 'learning_rate': 4.466463414634147e-05, 'epoch': 0.4268292682926829, 'total_flos': 14123752544489472, 'step': 3500}
{'loss': 2.60712

{'loss': 2.0251328125, 'learning_rate': 3.704268292682927e-05, 'epoch': 1.0365853658536586, 'total_flos': 34276911050651520, 'step': 8500}
{'loss': 1.987203125, 'learning_rate': 3.628048780487805e-05, 'epoch': 1.0975609756097562, 'total_flos': 36299134152746880, 'step': 9000}
{'loss': 1.970578125, 'learning_rate': 3.551829268292683e-05, 'epoch': 1.1585365853658536, 'total_flos': 38313950079125376, 'step': 9500}


KeyboardInterrupt: ignored

In [26]:
trainer.save_model("./martbertochar")

In [27]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./martbertochar",
    tokenizer="./martbertochar"
)

In [ ]:
tokenizer.mask_token

'<mask>'

In [42]:
fill_mask("अनेक सामाजिक चळवळीत, सत्याग्रहांमधे सहभाग घेतला.")

[{'score': 0.03394700214266777,
  'sequence': '<s>अनेक सामाजिक चळवळीत,०००मधे सहभाग घेतला.</s>',
  'token': 917,
  'token_str': 'à¥¦à¥¦à¥¦'},
 {'score': 0.013973863795399666,
  'sequence': '<s>अनेक सामाजिक चळवळीत, पणमधे सहभाग घेतला.</s>',
  'token': 753,
  'token_str': 'Ġà¤ªà¤£'},
 {'score': 0.01306256651878357,
  'sequence': '<s>अनेक सामाजिक चळवळीत,\xa0मधे सहभाग घेतला.</s>',
  'token': 353,
  'token_str': 'Âł'},
 {'score': 0.011251473799347878,
  'sequence': '<s>अनेक सामाजिक चळवळीत, मधे सहभाग घेतला.</s>',
  'token': 225,
  'token_str': 'Ġ'},
 {'score': 0.008268973790109158,
  'sequence': '<s>अनेक सामाजिक चळवळीत, ३मधे सहभाग घेतला.</s>',
  'token': 492,
  'token_str': 'Ġà¥©'}]

In [ ]:
 from tokenizers.decoders import ByteLevel
 decoder = ByteLevel()
 decoder.decode(['à¤¨' ])

'न'

In [ ]:
txt = "कोल्हापुरात मिळणारा तांबडा पांढरा <mask> कुठेच मिळत नाही."
enc = tokenizer.encode(txt)
print(tokenizer.decode(enc))

<s>कोल्हापुरात मिळणारा तांबडा पांढरा<mask> कुठेच मिळत नाही.</s>


In [ ]:
tokenizer.

SyntaxError: ignored